In [1]:
import tensorflow as tf
import numpy as np

tf.__version__

'1.15.0'

In [2]:
## GPU 메모리 관리 ##
#1번 방법
config = tf.ConfigProto()
config.gpu_options.allow_growth = True #메모리 수요에 따라 할당

'''주의할 점은, 이 옵션은 메모리의 증식만 가능하다는 것. 연산이 끝나고 메모리가 필요없는 상황이라고 해서 할당된 메모리를 반납하지 않는다. 

Tensorflow 측에서는 그럴 경우 더 심한 메모리 파편화를 일으킬 수도 있다고 하니 판단은 독자의 몫.'''

'주의할 점은, 이 옵션은 메모리의 증식만 가능하다는 것. 연산이 끝나고 메모리가 필요없는 상황이라고 해서 할당된 메모리를 반납하지 않는다. \n\nTensorflow 측에서는 그럴 경우 더 심한 메모리 파편화를 일으킬 수도 있다고 하니 판단은 독자의 몫.'

In [3]:
## GPU 메모리 관리 ##
#2번 방법
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3 #메모리 할당 비율 설정
#config.gpu_options.allow_growth = True #메모리 수요에 따라 할당, 두개 같이 사용 가능(추천하지는 않음)

In [4]:
##9.2 계산그래프를 만들어 세션에서 실행하기 ##
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

sess = tf.Session(config=config)
sess.run(x.initializer) #변수초기화
sess.run(y.initializer) #변수초기화
result = sess.run(f) #그래프평가(실행)

print(result)

sess.close()

42


In [5]:
## with를 사용한 방법 ##
with tf.Session(config=config) as sess:
    #x.initializer.run() #tf.get_default_graph().run(x.initializer)와 동일
    #y.initializer.run() #tf.get_default_graph().run(y.initializer)와 동일
    init = tf.global_variables_initializer()
    init.run() #모든 변수 초기화
    result = f.eval() #tf.get_default_graph().run(f)와 동일
    print(result)

42


In [6]:
sess = tf.InteractiveSession() #자기자신을 기본세션으로 지정
init.run()
result = f.eval()
print(result)
sess.close() #반드시 세션을 종료시켜야 함

42


In [7]:
## 9.3 계산 그래프 관리 ##
x1 = tf.Variable(1) #노드를 만들면 자동으로 기본 계산 그래프레 추가됨
x1.graph is tf.get_default_graph()

True

In [8]:
graph = tf.Graph() #독립적인 그래프 만들기
with graph.as_default(): #임시적으로 기본그래프로 사용
    x2 = tf.Variable(2)

print(x2.graph is graph) #독립적인 그래프에 속함
print(x2.graph is tf.get_default_graph()) #기본그래프에 속하지 않음

True
False


In [9]:
tf.reset_default_graph() #기본그래프 초기화, 중복된 노드 제거

In [10]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x + 3

with tf.Session() as sess:
    y_val, z_val = sess.run([y, z]) #변수는 세션이 유지되는 동안 유효함
    print(y_val)
    print(z_val)

10
8


In [3]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing("data/")
m, n = housing.data.shape
# bias 추가
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

print('data shape :', housing.data.shape)
print('data_plus_bias shape :', housing_data_plus_bias.shape)

data shape : (20640, 8)
data_plus_bias shape : (20640, 9)


<img src="https://render.githubusercontent.com/render/math?math=%5Chat%7B%5Cmathbf%7BW%7D%7D%20%3D%20%5Cleft%28%5Cmathbf%7BX%7D%5E%7BT%7D%20%5Cmathbf%7BX%7D%20%5Cright%29%5E%7B-1%7D%20%5Cmathbf%7BX%7D%5E%7BT%7D%20%5Cmathbf%7BY%7D&mode=display"/>

In [12]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
Y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="Y")
XT = tf.transpose(X) #전치행렬로 만듬(행과 열을 바꿈)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), Y) #정규방정식

with tf.Session(config=config) as sess:
    theta_value = theta.eval()
    
theta_value

array([[-3.68962631e+01],
       [ 4.36777472e-01],
       [ 9.44449380e-03],
       [-1.07348785e-01],
       [ 6.44962370e-01],
       [-3.94082872e-06],
       [-3.78797273e-03],
       [-4.20847952e-01],
       [-4.34020907e-01]], dtype=float32)

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

In [14]:
## 9.6 경사하강법 구현 ##
## 9.6.1 직접 그래디언트 계산 ##
n_epochs = 10000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
Y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="Y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta") #난수를 담은 텐서를 생성하는 노드를 그래프에 생성
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - Y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients) #변수에 새로운 값을 할당하는 노드 생성

init = tf.global_variables_initializer()

with tf.Session(config=config) as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 1000 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()
    
best_theta

Epoch 0 MSE =  8.802726
Epoch 1000 MSE =  0.53485996
Epoch 2000 MSE =  0.5253611
Epoch 3000 MSE =  0.52446365
Epoch 4000 MSE =  0.5243431
Epoch 5000 MSE =  0.5243245
Epoch 6000 MSE =  0.52432156
Epoch 7000 MSE =  0.524321
Epoch 8000 MSE =  0.524321
Epoch 9000 MSE =  0.524321


array([[ 2.0685523 ],
       [ 0.82964474],
       [ 0.11875628],
       [-0.2655744 ],
       [ 0.3057353 ],
       [-0.00450153],
       [-0.03932726],
       [-0.89982766],
       [-0.870487  ]], dtype=float32)

In [15]:
## 9.6.2 자동미분사용 ##
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
Y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="Y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta") #난수를 담은 텐서를 생성하는 노드를 그래프에 생성
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - Y
mse = tf.reduce_mean(tf.square(error), name="mse")
#gradients = 2/m * tf.matmul(tf.transpose(X), error)
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients) #변수에 새로운 값을 할당하는 노드 생성

init = tf.global_variables_initializer()

with tf.Session(config=config) as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

best_theta

Epoch 0 MSE =  11.879491
Epoch 100 MSE =  0.75746316
Epoch 200 MSE =  0.57445425
Epoch 300 MSE =  0.55828637
Epoch 400 MSE =  0.54878676
Epoch 500 MSE =  0.54198045
Epoch 600 MSE =  0.5370703
Epoch 700 MSE =  0.5335268
Epoch 800 MSE =  0.530969
Epoch 900 MSE =  0.5291229


array([[ 2.0685523 ],
       [ 0.8021441 ],
       [ 0.13669826],
       [-0.17041533],
       [ 0.20804848],
       [ 0.00238259],
       [-0.04025865],
       [-0.79486364],
       [-0.7602312 ]], dtype=float32)

In [16]:
## 9.6.3 옵티마이저 사용 ##
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
Y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="Y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta") #난수를 담은 텐서를 생성하는 노드를 그래프에 생성
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - Y
mse = tf.reduce_mean(tf.square(error), name="mse")
#gradients = tf.gradients(mse, [theta])[0]
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
#training_op = tf.assign(theta, theta - learning_rate * gradients) #변수에 새로운 값을 할당하는 노드 생성
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session(config=config) as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()
    
best_theta

Epoch 0 MSE =  9.297094
Epoch 100 MSE =  0.84140295
Epoch 200 MSE =  0.6377024
Epoch 300 MSE =  0.6112633
Epoch 400 MSE =  0.5950269
Epoch 500 MSE =  0.58218277
Epoch 600 MSE =  0.5717872
Epoch 700 MSE =  0.56333417
Epoch 800 MSE =  0.55644065
Epoch 900 MSE =  0.5508048


array([[ 2.0685523 ],
       [ 0.97097486],
       [ 0.15910296],
       [-0.5066931 ],
       [ 0.4937275 ],
       [ 0.00834046],
       [-0.04575343],
       [-0.47298685],
       [-0.45884115]], dtype=float32)

In [21]:
## 9.7 미니배치 경사하강법 구현 ##
n_epochs = 50
learning_rate = 0.0001
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n+1), name="X") #차원이 None이면 어떤크기도 가능하다는 뜻
Y = tf.placeholder(tf.float32, shape=(None, 1), name="Y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta") #난수를 담은 텐서를 생성하는 노드를 그래프에 생성
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - Y
mse = tf.reduce_mean(tf.square(error), name="mse")
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9) #다른 옵티마이저 사용
training_op = optimizer.minimize(mse)

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    Y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, Y_batch

init = tf.global_variables_initializer()

with tf.Session(config=config) as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, Y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X : X_batch, Y : Y_batch}) #placeholder에 의존하는 식을 실행하는데 값을 주지않으면 예외발생
        if epoch % 10 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval(feed_dict={X : X_batch, Y : Y_batch}))
    
    best_theta = theta.eval()
    
best_theta

Epoch 0 MSE =  1.6304214
Epoch 10 MSE =  0.6231211
Epoch 20 MSE =  0.53415525
Epoch 30 MSE =  0.61872923
Epoch 40 MSE =  0.34299383


array([[ 2.0676532 ],
       [ 0.81094784],
       [ 0.13473205],
       [-0.1928155 ],
       [ 0.2533899 ],
       [ 0.00396125],
       [-0.04528483],
       [-0.82152164],
       [-0.78731716]], dtype=float32)

In [6]:
## 9.8 모델 저장과 복원 ##
n_epochs = 500
learning_rate = 0.01
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

tf.reset_default_graph()

X = tf.placeholder(tf.float32, shape=(None, n+1), name="X") #차원이 None이면 어떤크기도 가능하다는 뜻
Y = tf.placeholder(tf.float32, shape=(None, 1), name="Y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta") #난수를 담은 텐서를 생성하는 노드를 그래프에 생성
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - Y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    Y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, Y_batch

init = tf.global_variables_initializer()
#saver = tf.train.Saver() #모델저장 노드 추가
#saver = tf.train.Saver({"weights" : theta}) #theta변수만 weight라는 이름으로 저장
saver = tf.train.Saver(max_to_keep=3)

#파라메터 저장#
tf.add_to_collection('theta', theta)
###############

with tf.Session(config=config) as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, Y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X : X_batch, Y : Y_batch}) #placeholder에 의존하는 식을 실행하는데 값을 주지않으면 예외발생
        if epoch % 100 == 0:
            save_path = saver.save(sess, "data/model_save.ckpt") #모델저장
            print("Epoch", epoch, "MSE = ", mse.eval(feed_dict={X : X_batch, Y : Y_batch}))
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "data/final_model_save.ckpt") #모델저장
    
best_theta

Epoch 0 MSE =  0.5342791
Epoch 100 MSE =  0.59744805
Epoch 200 MSE =  0.5149244
Epoch 300 MSE =  0.58632696
Epoch 400 MSE =  0.41618687


array([[ 2.0626614 ],
       [ 0.83224726],
       [ 0.11040603],
       [-0.27255896],
       [ 0.25648004],
       [-0.00795209],
       [-0.07186626],
       [-0.8915991 ],
       [-0.8733595 ]], dtype=float32)

In [23]:
## 9.8 모델 저장과 복원 방법1 ##
n_epochs = 100
learning_rate = 0.001
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

tf.reset_default_graph()

with tf.Session() as sess:
    global theta
    saver = tf.train.import_meta_graph("data/final_model_save.ckpt.meta") #meta파일에 저장된 그래프 구조 복원
    saver.restore(sess, 'data/final_model_save.ckpt')
    theta = tf.get_collection('theta')[0] #파라메터 복원
    #theta = tf.get_default_graph().get_tensor_by_name("theta:0")

X = tf.placeholder(tf.float32, shape=(None, n+1), name="X") #차원이 None이면 어떤크기도 가능하다는 뜻
Y = tf.placeholder(tf.float32, shape=(None, 1), name="Y")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - Y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
saver = tf.train.import_meta_graph("data/final_model_save.ckpt.meta") #meta파일에 저장된 그래프 구조 복원

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    Y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, Y_batch

with tf.Session(config=config) as sess:
    saver.restore(sess, "data/final_model_save.ckpt") #저장한 모델 불러오기
    print(theta.eval())
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, Y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X : X_batch, Y : Y_batch}) #placeholder에 의존하는 식을 실행하는데 값을 주지않으면 예외발생
        if epoch % 10 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval(feed_dict={X : X_batch, Y : Y_batch}))
    
    best_theta = theta.eval()
    
best_theta

INFO:tensorflow:Restoring parameters from data/final_model_save.ckpt
INFO:tensorflow:Restoring parameters from data/final_model_save.ckpt
[[ 2.0544672 ]
 [ 0.82970107]
 [ 0.10713524]
 [-0.31074822]
 [ 0.24564512]
 [-0.00222411]
 [-0.01061386]
 [-0.891216  ]
 [-0.8752481 ]]
Epoch 0 MSE =  0.49956453
Epoch 10 MSE =  0.5664362
Epoch 20 MSE =  0.5255838
Epoch 30 MSE =  0.6023059
Epoch 40 MSE =  0.34531453
Epoch 50 MSE =  0.44214913
Epoch 60 MSE =  0.5371403
Epoch 70 MSE =  0.4436193
Epoch 80 MSE =  0.5882714
Epoch 90 MSE =  0.3841381


array([[ 2.0674198 ],
       [ 0.83073413],
       [ 0.11769314],
       [-0.27660346],
       [ 0.29551786],
       [-0.00227595],
       [-0.0368447 ],
       [-0.89784104],
       [-0.8702798 ]], dtype=float32)

In [14]:
## 모델 복원 방법2 ##
n_epochs = 100
learning_rate = 0.001
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

tf.reset_default_graph()

saver = tf.train.import_meta_graph("data/final_model_save.ckpt.meta") #meta파일에 저장된 그래프 구조 복원, 복원된 그래프를 기본그래프에 추가 후 Saver객체 반환
theta = tf.get_default_graph().get_tensor_by_name("theta:0")
X = tf.placeholder(tf.float32, shape=(None, n+1), name="X") #차원이 None이면 어떤크기도 가능하다는 뜻
Y = tf.placeholder(tf.float32, shape=(None, 1), name="Y")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - Y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)
    indices = np.random.randint(m, size=batch_size)
    X_batch = scaled_housing_data_plus_bias[indices]
    Y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, Y_batch

with tf.Session(config=config) as sess:
    saver.restore(sess, "data/final_model_save.ckpt") #저장한 모델 불러오기
    print(sess.run(theta, feed_dict={X : X_batch, Y : Y_batch})) #theta의 값 출력
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, Y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X : X_batch, Y : Y_batch}) #placeholder에 의존하는 식을 실행하는데 값을 주지않으면 예외발생
        if epoch % 10 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval(feed_dict={X : X_batch, Y : Y_batch}))
    
    best_theta = theta.eval()
    
best_theta

INFO:tensorflow:Restoring parameters from data/final_model_save.ckpt
[[ 2.0544672 ]
 [ 0.82970107]
 [ 0.10713524]
 [-0.31074822]
 [ 0.24564512]
 [-0.00222411]
 [-0.01061386]
 [-0.891216  ]
 [-0.8752481 ]]
Epoch 0 MSE =  0.49956453
Epoch 10 MSE =  0.5664362
Epoch 20 MSE =  0.5255838
Epoch 30 MSE =  0.6023059
Epoch 40 MSE =  0.34531453
Epoch 50 MSE =  0.44214913
Epoch 60 MSE =  0.5371403
Epoch 70 MSE =  0.4436193
Epoch 80 MSE =  0.5882714
Epoch 90 MSE =  0.3841381


array([[ 2.0674198 ],
       [ 0.83073413],
       [ 0.11769314],
       [-0.27660346],
       [ 0.29551786],
       [-0.00227595],
       [-0.0368447 ],
       [-0.89784104],
       [-0.8702798 ]], dtype=float32)

In [12]:
## 9.9 텐서보드로 시각화 하기 ##
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf.logs"
logdir = f"{root_logdir}/run-{now}"

#### 트레이닝 ####
tf.reset_default_graph()

n_epochs = 10000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
Y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="Y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta") #난수를 담은 텐서를 생성하는 노드를 그래프에 생성
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - Y
mse = tf.reduce_mean(tf.square(error), name="mse")
#gradients = tf.gradients(mse, [theta])[0]
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
#training_op = tf.assign(theta, theta - learning_rate * gradients) #변수에 새로운 값을 할당하는 노드 생성
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
mse_summary = tf.summary.scalar('MSE', mse) #MSE값을 평가하고 텐서보드가 인식하는 이진 로그 문자열에 쓰기 위한 노드(서머리)를 추가
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph()) #기록할 로그파일, 기록할 그래프 

with tf.Session(config=config) as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 1000 == 0:
            summary_str = mse_summary.eval()
            step = epoch * n_batches + batch_index
            file_writer.add_summary(summary_str, step)
            print("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()
##########################################################################################################################################
file_writer.close()

#tensorboard --logdir [이벤트파일경로] 실행
#http://localhost:6006/#scalars로 접속

Epoch 0 MSE =  7.619415
Epoch 1000 MSE =  0.5514102
Epoch 2000 MSE =  0.5269976
Epoch 3000 MSE =  0.5246882
Epoch 4000 MSE =  0.5243778
Epoch 5000 MSE =  0.5243301
Epoch 6000 MSE =  0.52432245
Epoch 7000 MSE =  0.52432126
Epoch 8000 MSE =  0.524321
Epoch 9000 MSE =  0.524321


In [14]:
## 9.10 이름범위 ##
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf.logs"
logdir = f"{root_logdir}/run-{now}"

#### 트레이닝 ####
tf.reset_default_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
Y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="Y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta") #난수를 담은 텐서를 생성하는 노드를 그래프에 생성
y_pred = tf.matmul(X, theta, name="predictions")

####이름범위를 만들어서 관련노드들을 그룹으로 묶음####
with tf.name_scope("loss") as scope:
    error = y_pred - Y
    mse = tf.reduce_mean(tf.square(error), name="mse")
######################################################

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
mse_summary = tf.summary.scalar('MSE', mse) #MSE값을 평가하고 텐서보드가 인식하는 이진 로그 문자열에 쓰기 위한 노드(서머리)를 추가
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph()) #기록할 로그파일, 기록할 그래프 

with tf.Session(config=config) as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            summary_str = mse_summary.eval()
            step = epoch * n_batches + batch_index
            file_writer.add_summary(summary_str, step)
            print("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()
##########################################################################################################################################
file_writer.close()

print(error.op.name) #범위 내에 있는 모든 연산의 이름에는 "loss/"라는 접두사가 붙음
print(mse.op.name) #범위 내에 있는 모든 연산의 이름에는 "loss/"라는 접두사가 붙음

Epoch 0 MSE =  14.490098
Epoch 100 MSE =  1.0712796
Epoch 200 MSE =  0.7618898
Epoch 300 MSE =  0.69183916
Epoch 400 MSE =  0.6450376
Epoch 500 MSE =  0.61141586
Epoch 600 MSE =  0.5871671
Epoch 700 MSE =  0.56967264
Epoch 800 MSE =  0.55705035
Epoch 900 MSE =  0.5479429
loss/sub
loss/mse


In [9]:
## 9.11 모듈화 ##

def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, 0., name="relu")

tf.reset_default_graph()
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for _ in range(5)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("tf.logs/relu", tf.get_default_graph()) #그래프저장
file_writer.close()

In [10]:
## 이름번위를 사용한 모듈화 ##

def relu(X):
    with tf.name_scope("relu"): #더 간단하게 표시
        w_shape = (int(X.get_shape()[1]), 1)
        w = tf.Variable(tf.random_normal(w_shape), name="weights")
        b = tf.Variable(0.0, name="bias")
        z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, 0., name="relu")

tf.reset_default_graph()
n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for _ in range(5)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("tf.logs/relu2", tf.get_default_graph()) #그래프저장
file_writer.close()

In [11]:
## 9.12 변수공유 ##

def relu(X):
    with tf.variable_scope("relu", reuse=True) as scope: #변수를 재사용하기 위해 resue를 True로 설정
        #scope.reuse_variables() #변수 재사용하기
        threshold = tf.get_variable("threshold")
        w_shape = int(X.get_shape()[1]), 1                          # not shown
        w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
        b = tf.Variable(0.0, name="bias")                           # not shown
        z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
        return tf.maximum(z, threshold, name="max")

tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
with tf.variable_scope("relu"): #이름범위 사용, threshold변수가 함수 밖에서 선언됨
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0)) #공유변수가 있다면 얻어옴, 없으면 생성
relus = [relu(X) for _ in range(5)]
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("tf.logs/relu3", tf.get_default_graph()) #그래프저장
file_writer.close()

In [14]:
def relu(X):
    #threshold변수가 함수 내부에서 선언됨
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0)) #공유변수가 있다면 얻어옴, 없으면 생성
    w_shape = int(X.get_shape()[1]), 1                          # not shown
    w = tf.Variable(tf.random_normal(w_shape), name="weights")  # not shown
    b = tf.Variable(0.0, name="bias")                           # not shown
    z = tf.add(tf.matmul(X, w), b, name="z")                    # not shown
    return tf.maximum(z, threshold, name="max")

tf.reset_default_graph()
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = []
for relu_index in range(5):
    with tf.variable_scope("relu", reuse=(relu_index >= 1)) as scope: #relu_index가 1보다 크면 재사용
        relus.append(relu(X))
output = tf.add_n(relus, name="output")

file_writer = tf.summary.FileWriter("tf.logs/relu4", tf.get_default_graph()) #그래프저장
file_writer.close()